<a href="https://colab.research.google.com/github/kittimaxz/Project_BoneAge/blob/main/Model_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
# python libraties
import os, cv2,itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from glob import glob
from PIL import Image
import scipy.stats as stats
# pytorch libraries
import torch
from torch import optim,nn
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset
from torchvision import models,transforms
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

In [2]:
from google.colab import drive # เชื่อม drive ของเรา ถ้าเชื่อมสำเร็จจะขึ้นคำว่าMounted at /content/drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os # os จัดการเกี่ยวกับไฟล์ต่างๆ ในโฟล์เดอร์

In [4]:
path = '/content/drive/My Drive/Project_Boneage'  # ชี้ว่า data เราอยู่โฟล์เดอร์ไหน

# ***Some Setup***

The cell below creates the pandas dataframes for training and testing.

In [5]:
#loading dataframes # เอาที่อยู่ของไฟล์เชื่อมกับที่อยู่ของไฟล์เรา โดยกำหนดชื่่อไฟล์ที่เราต้องการ
train = pd.read_csv(r"/content/drive/MyDrive/Project_Boneage/train.csv")
validation = pd.read_csv(r"/content/drive/MyDrive/Project_Boneage/validation.csv")
test = pd.read_csv(r"/content/drive/MyDrive/Project_Boneage/boneage-test-dataset.csv")

In [6]:
train_img = '/content/drive/My Drive/Project_Boneage/boneage_training_dataset/'
val_img = '/content/drive/My Drive/Project_Boneage/boneage_validation_dataset/'
test_img = '/content/drive/My Drive/Project_Boneage/boneage-test-dataset/'

In [7]:
train # แสดง data 'train'

,id,boneage,male
0,1377,180,False
1,1378,12,False
2,1379,94,False
3,1380,120,True
4,1381,82,False
...,...,...,...
10084,13259,94,False
10085,13261,94,False
10086,13274,94,False
10087,13286,94,False


In [8]:
validation # แสดง data 'validation'

,id,boneage,male
0,10186,10,True
1,10758,15,False
2,10860,180,False
3,10877,180,False
4,11008,180,False
...,...,...,...
2517,15605,50,False
2518,15606,113,False
2519,15608,55,False
2520,15609,150,True


In [9]:
test # แสดง data 'test'

,Case ID,Sex,boneage
0,4360,M,169
1,4361,M,170
2,4362,M,73
3,4363,M,153
4,4364,M,135
...,...,...,...
195,4555,F,133
196,4556,F,129
197,4557,F,168
198,4558,F,136


In [10]:
#appending file extension to id column for both training and testing dataframes
# ต่อท้าย data ใน column 'id' ด้วย .png
train['id'] = train['id'].apply(lambda x: str(x)+'.png') # ทำการเปลี่ยน type ใน column 'id' จาก int  เป็น string เพื่อให้สามารถต่อท้ายด้วย .png ได้ # ฟังก์ชันแลมบ์ดาเป็นฟังก์ชันที่ไม่ระบุตัวตนขนาดเล็ก
validation['id'] = validation['id'].apply(lambda x: str(x)+'.png')
test['Case ID'] = test['Case ID'].apply(lambda x: str(x)+'.png')

train.head() # แสดงค่าของ trainบางส่วน

,id,boneage,male
0,1377.png,180,False
1,1378.png,12,False
2,1379.png,94,False
3,1380.png,120,True
4,1381.png,82,False


In [11]:
train['gender'] = train['male'].apply(lambda x: '0' if x else '1') # ทำการเปลี่ยนค่าข้อมูล gender จาก true -> male, false -> female 
train['gender']

0        1
1        1
2        1
3        0
4        1
        ..
10084    1
10085    1
10086    1
10087    1
10088    1
Name: gender, Length: 10089, dtype: object

In [12]:
validation['gender'] = validation['male'].apply(lambda x: '0' if x else '1') # ทำการเปลี่ยนค่าข้อมูล gender จาก true -> male, false -> female 
validation['gender']

0       0
1       1
2       1
3       1
4       1
       ..
2517    1
2518    1
2519    1
2520    0
2521    0
Name: gender, Length: 2522, dtype: object

In [13]:
train['gender'] = train['gender'].apply(lambda x: str(x)) 
validation['gender'] = validation['gender'].apply(lambda x: str(x))

In [14]:
print(train['gender'].value_counts()) #แสดงจำนวนเพศหญิงและเพศชายทั้งหมดใน trian_df

0    5468
1    4621
Name: gender, dtype: int64


In [15]:
train.shape # ข้อมูลมีทั้งหมด 261 จากทั้งหมด 4 column

(10089, 4)

In [16]:
class ImageDataset(Dataset):

    def __init__(self, pickle_file, image_path):
        self.image_dir = image_path
        self.pickle_file = pickle_file
        self.labels = pd.read_csv(pickle_file)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        labels = self.labels.iloc[idx, 0:]
        images = Image.open(f"{self.image_dir}{labels['id']}.png")
        images = np.array(images)
        #image = image[..., :3]

        images = transforms.functional.to_tensor(images)
        images = transforms.functional.resize(images, size=[500,500])
        return images

In [17]:
image_datatrain = ImageDataset(path+"/train.csv", image_path=f"{train_img}")
image_datatrain

In [18]:
image_dataval = ImageDataset(path+"/validation.csv", image_path=f"{val_img}")
image_dataval

In [19]:
image_datatest = ImageDataset(path+"/boneage-test-dataset.csv", image_path=f"{test_img}")
image_datatest

In [20]:
strat_epoch =0 
num_epochs=60
lr=0.0001
patience=2 
batch_size=20
num_workers=0

In [21]:
train_dataloader = DataLoader(image_datatrain, batch_size,num_workers)
train_dataloader

In [22]:
val_dataloader = DataLoader(image_dataval, batch_size,num_workers)
val_dataloader

In [23]:
test_dataloader = DataLoader(image_datatest, batch_size,num_workers)
test_dataloader

In [24]:
! git clone https://github.com/BCV-Uniandes/Bonet.git
!cd Bonet

Cloning into 'Bonet'...
remote: Enumerating objects: 52, done.
remote: Total 52 (delta 0), reused 0 (delta 0), pack-reused 52
Unpacking objects: 100% (52/52), 136.62 KiB | 1.77 MiB/s, done.


In [25]:
# Local imports
from Bonet.models.backbone.inception_v3 import InceptionV3

In [26]:
class BoNetModule(nn.Module):

    def __init__(
        self, transform_input = False, lr = 0.0001, num_workers = 0, batch_size =20
    ):

        super().__init__()
        self.lr = lr
        self.num_workers = num_workers
        self.batch_size = batch_size
 
        # Backbone
        self.inception_v3 = InceptionV3()
        self.fc_1 = DenseLayer(100384, 1000)
        self.fc_2 = DenseLayer(1000, 1000)
        self.fc_3 = nn.Linear(1000, 1)

    def forward(self, x):
        x = self.inception_v3(x)
        x = self.fc_1(torch.cat([x], 1))
        x = self.fc_2(x)
        x = self.fc_3(x)
        return x

    def training_step(self, batch, batch_idx):
      images, labels = batch

      criterion = torch.nn.L1Loss()
      y_pred = torch.flatten(self(images, labels))
      y_pred = y_pred.double()

      loss = criterion(y_pred)
      return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        images, labels = batch

        criterion = torch.nn.L1Loss()
        y_pred = torch.flatten(self(images, labels))
        y_pred = y_pred.double()

        val_loss = criterion(y_pred)

        return {"val_loss": val_loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        return {"val_loss": avg_loss}

    def test_step(self, batch, batch_idx):
        images, labels = batch

        criterion = torch.nn.L1Loss()
        y_pred = torch.flatten(self(images, labels))
        y_pred = y_pred.double()

        test_loss = criterion(y_pred)

        return {"test_loss": test_loss}

    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        logs = {"test_loss": avg_loss}
        return {"test_loss": avg_loss, "log": logs, "progress_bar": logs}

    def setup(self, stage):

        image_datatrain = ImageDataset(pickle_file=path+"/train.csv", image_dir=f"{train_img}")
        image_dataval = ImageDataset(pickle_file=path+"/validation.csv", image_dir=f"{val_img}")
        image_datatest = ImageDataset(pickle_file=path+"/boneage-test-dataset.csv", image_dir=f"{test_img}")

        #train_size = int(0.80 * len(image_data))
        #val_size = int((len(image_data) - train_size) / 2)
        #test_size = int((len(image_data) - train_size) / 2)

        #self.train_set, self.val_set, self.test_set = random_split(image_data, (train_size, val_size, test_size))
        self.train_set = image_datatrain
        self.val_set = image_dataval
        self.test_set = image_datatest

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size)
    
class DenseLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.linear = nn.Linear(in_channels, out_channels)

    def forward(self, x):
        x = self.linear(x)
        return F.relu(x, inplace=True)

In [27]:
model = BoNetModule()

In [28]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr)

In [29]:
print(model)

BoNetModule(
  (inception_v3): InceptionV3(
    (Conv2d_1a_3x3): BasicConv2d(
      (conv): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2a_3x3): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2b_3x3): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_3b_1x1): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_4a_3x3): BasicConv2d(
      (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False

In [30]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i in enumerate(train_dataloader):
        # Every data instance is an input + label pair
        inputs, labels = train_dataloader

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(train_dataloader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

In [32]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/boneage_trainer_{}'.format(timestamp))

best_vloss = 1_000_000.

for epoch in range(num_epochs):
    print('num_epochs {}:'.format(strat_epoch + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(strat_epoch, writer)

    # We don't need gradients on to do reporting
    model.train(False)

    val_loss = 0.0
    for i, valdation in enumerate(val_dataloader):
        inputs, labels = valdation
        outputs = model(inputs)
        val_loss = loss_fn(outputs, labels)
        val_loss += val_loss

    avg_vloss = val_loss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    strat_epoch + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, strat_epoch)
        torch.save(model.state_dict(), model_path)

    strat_epoch += 1

num_epochs 1:


NameError: ignored